実行環境 <br>
- Python 3.11.3
- Pandas 2.0.2
- Numpy 1.24.3

In [2]:
import pandas as pd
import numpy as np

# 物件情報

In [64]:
property_info = pd.read_csv('./費用情報.csv',dtype='object')
property_info_0 = property_info.copy()

In [65]:
property_info = property_info_0.copy()

# '○万円' -> int の変換
def obj_to_int(col):
  property_info[col].replace('-','000', inplace=True)
  property_info[col] = property_info[col].str[:-2].astype(float)
  property_info[col] = property_info[col]*10000
  property_info[col] = property_info[col].astype(pd.Int64Dtype())
  property_info[col].replace(0,np.NAN, inplace=True)
  return property_info

# 賃料：object -> int
property_info = obj_to_int('賃料')

# 敷金 object -> int
property_info = obj_to_int('敷金')

# 礼金 object -> int
property_info = obj_to_int('礼金')

# 保証金 object -> int
property_info = obj_to_int('保証金')

# 敷引・償却 object -> int
property_info = obj_to_int('敷引・償却')

# 管理費 object -> int 
property_info['管理費'] = property_info['管理費'].str[:-1].astype(int)


property_info.head()

,物件CD,賃料,管理費,敷金,礼金,保証金,敷引・償却
0,100343313667,69000,10000,<NA>,69000,<NA>,<NA>
1,100343314105,69000,10000,<NA>,69000,<NA>,<NA>
2,100343313391,70000,10000,<NA>,70000,<NA>,<NA>
3,100343842510,70000,5000,70000,<NA>,<NA>,<NA>
4,100343709215,70000,5000,70000,<NA>,<NA>,<NA>


# 物件基本情報

In [66]:
property_basic_info = pd.read_csv('./物件基本情報.csv',dtype='object')
property_basic_info_0 = property_basic_info.copy()

In [146]:
property_basic_info = property_basic_info_0.copy()

# 物件名 -> 物件名, 不動産会社
property_basic_info['物件名'] = property_basic_info['物件名'].str.split(' - ')
property_basic_info['不動産'] = property_basic_info['物件名'].str[1]
property_basic_info['不動産'] = property_basic_info['不動産'].str.replace('が提供する賃貸物件情報','')
property_basic_info['物件名'] = property_basic_info['物件名'].str[0]

# 所在地を 都道府県, 市区町村, 番地 に分割
def div_plc(a,b,c,d):
  div_col = property_basic_info['所在地'].apply(
    lambda x: 
      x[:x.find(a)+1] if x.find(a) > 0 
      else x[:x.find(b)+1] if x.find(b) > 0 
      else x[:x.find(c)+1] if x.find(c) > 0 
      else x[:x.find(d)+1]
  )
  return div_col
# 都道府県
property_basic_info['所在地_都道府県'] = div_plc('都','道','府','県')
# 市区町村
property_basic_info['所在地'] = property_basic_info.apply(
  lambda x:
    x['所在地'].replace(x['所在地_都道府県'],'') ,axis=1
  )
property_basic_info['所在地_市区町村'] = div_plc('市','区','町','村')
# 番地
property_basic_info['所在地_番地'] = property_basic_info.apply(
  lambda x:
    x['所在地'].replace(x['所在地_市区町村'],'') ,axis=1
  )
# 処理前の列を削除
property_basic_info.drop(columns='所在地',inplace=True)

# 階建 
floor_story = property_basic_info['階建'].str.split('/')
property_basic_info['階'] = floor_story.str[0].str[:-1].astype(int)
property_basic_info['階建'] = floor_story.str[1].str[:-2].astype(int)

property_basic_info = property_basic_info[[
  '物件CD', '物件名','築年数', '所在地_都道府県', '所在地_市区町村', '所在地_番地',
  '建築種別', '構造', '向き', '階建', '階', '間取り', '間取り詳細', '専有面積', '間取り図',
  '取引様態', '取引先', '更新日', '不動産'
  ]]


In [145]:
property_basic_info_0.columns

Index(['物件CD', '物件名', '所在地', '築年数', '建築種別', '構造', '向き', '階建', '間取り', '間取り詳細',
       '専有面積', '間取り図', '取引様態', '取引先', '更新日'],
      dtype='object')

In [149]:
property_basic_info = property_basic_info_0.copy()




property_basic_info['間取り詳細'].value_counts()




間取り詳細
-            14
洋6.2         11
洋6.5         10
洋6 K1.5       7
洋6.1          7
洋6.2 K2       5
K             5
洋7.8          4
洋6.4          4
洋6.7          4
洋6            4
洋6.1 K1.5     4
洋6 K2         3
洋6.4 K2.0     3
洋7.9 K2       2
K2.0          2
洋7 K1.5       2
洋6.2 K2.0     2
洋6.4 K1.5     1
洋6.1 K2       1
洋7            1
洋7.9          1
洋6.4 K        1
洋6.5 K2       1
洋10.7         1
洋6.7 K        1
洋6.4 K2       1
Name: count, dtype: int64

In [144]:
property_basic_info.columns

Index(['物件CD', '築年数', '建築種別', '構造', '向き', '階建', '間取り', '間取り詳細', '専有面積', '間取り図',
       '取引様態', '取引先', '更新日', '不動産', '所在地_都道府県', '所在地_市区町村', '所在地_番地', '階'],
      dtype='object')

In [147]:
property_basic_info

,物件CD,物件名,築年数,所在地_都道府県,所在地_市区町村,所在地_番地,建築種別,構造,向き,階建,階,間取り,間取り詳細,専有面積,間取り図,取引様態,取引先,更新日,不動産
0,100343313667,リヴシティ横浜東ベイサイド,2010年11月,神奈川県,横浜市,神奈川区東神奈川２,マンション,鉄筋コン,南東,10,6,1K,洋6.7,20.02m2,https://img01.suumo.com/front/gazo/fr/bukken/6...,仲介,(株)タウンハウジング神奈川 大倉山店,2023/09/12,(株)タウンハウジング神奈川 大倉山店
1,100343314105,リヴシティ横浜東ベイサイド,2010年11月,神奈川県,横浜市,神奈川区東神奈川２,マンション,鉄筋コン,南東,10,6,1K,洋6.7,20.02m2,https://img01.suumo.com/front/gazo/fr/bukken/1...,仲介,(株)タウンハウジング神奈川 鶴見店,2023/09/13,(株)タウンハウジング神奈川 鶴見店
2,100343313391,リヴシティ横浜東ベイサイド,2010年11月,神奈川県,横浜市,神奈川区東神奈川２,マンション,鉄筋コン,南東,10,6,1K,-,20.02m2,https://img01.suumo.com/front/gazo/fr/bukken/3...,仲介,(株)タウンハウジング神奈川 横浜店,2023/09/12,(株)タウンハウジング神奈川 横浜店
3,100343842510,マリーナヨコハマベイサイド 203号室,2017年6月,神奈川県,横浜市,神奈川区東神奈川２,マンション,鉄筋コン,南東,5,2,1K,洋6.7,20.18m2,https://img01.suumo.com/front/gazo/fr/bukken/5...,仲介,(株)未来企画桜木町店,2023/09/12,(株)未来企画桜木町店
4,100343709215,マリーナヨコハマベイサイド 203号室,2017年6月,神奈川県,横浜市,神奈川区東神奈川２,マンション,鉄筋コン,南東,5,2,1K,洋6.7,20.18m2,https://img01.suumo.com/front/gazo/fr/bukken/2...,仲介,(株)未来企画本店,2023/09/12,(株)未来企画本店
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,100336530692,リヴシティ横濱七島町 604号室,2017年7月,神奈川県,横浜市,神奈川区七島町,マンション,鉄筋コン,南西,7,6,1K,-,22.1m2,https://img01.suumo.com/front/gazo/fr/bukken/6...,仲介,(株)アンビション・バロー川崎店,2023/09/12,(株)アンビション・バロー川崎店
98,100341277604,京急本線 子安駅 7階建 築7年,2017年7月,神奈川県,横浜市,神奈川区七島町,マンション,鉄筋コン,北東,7,2,1K,-,21.7m2,https://img01.suumo.com/front/gazo/fr/bukken/6...,仲介,(株)アンビション・バロー新横浜店,2023/09/12,(株)アンビション・バロー新横浜店
99,100333151536,京急本線 子安駅 7階建 築7年,2017年7月,神奈川県,横浜市,神奈川区七島町,マンション,鉄筋コン,南西,7,7,1K,洋6.4,22.11m2,https://img01.suumo.com/front/gazo/fr/bukken/5...,仲介,(株)ネクストライフ横浜北幸店,2023/09/12,(株)ネクストライフ横浜北幸店
100,100342297202,京急本線 子安駅 7階建 築7年,2017年7月,神奈川県,横浜市,神奈川区七島町,マンション,鉄筋コン,南西,7,7,1K,洋6.4,22.11m2,https://img01.suumo.com/front/gazo/fr/bukken/2...,仲介,(株)ネクストライフ横浜鶴屋町店,2023/09/12,(株)ネクストライフ横浜鶴屋町店
